# Request VC from Authority

This notebook is used in combination with notebook `01_issue_VC_city.ipynb` (see Authority agent). Break points indicate when to switch notebooks.

In [1]:
%%javascript
document.title ='Manufacturer1 Agent'

<IPython.core.display.Javascript object>

#### Imports

In [2]:
#%autoawait 
from aries_cloudcontroller import AriesAgentController
from libs.connection_service import ConnectionService
import libs.functions as fnc
import os
#from pprintpp import pprint
#from termcolor import colored

#### Initialize `Manufacturer1` Agent Controller

In [3]:
# Get relevant details from .env file
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

In [4]:
# Setup
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Initialising a controller with admin api at http://manufacturer1-agent:3021 and an api key of adminApiKey
Listening for webhooks from agent at http://0.0.0.0:3010


In [5]:
connections = ConnectionService(agent_controller, role="holder")

Successfully initiated connection handler for holder ACA-PY agent


## 1 – Init Webhook Server and register Event Listeners for it
#### 1.1 – Start webhook server
Start a webhook server to be able to communicate with other agents

In [ ]:
# Check if Manufacturer already has a VC
connections.get_credentials()

#### 1.2 – Define listeners
Define listeners that are triggered when something happens on the webhook server

#### 1.3 – Register listeners with `agent_controller`

## 2 – Establish a connection with Authority agent
A connection with the credential issuer (i.e., the authority agent) must be establieshed before a VC can be received. In this scenario, the agent requests a connection with the Authority to be certified as an official city agency. Thus, the city agent sends an invitation to the Authority.

### 2.1 Create invitation to Authority agent

In [ ]:
# Setup for connection with Authority agent
alias = None
auto_accept = "true" # Accept response of Authority agent right away
public = "false" # Do not use public DID
multi_use = "false" # Invitation is only for one invitee

connection_id = connections.create_connection_invitation(alias=alias, auto_accept=auto_accept, public=public, multi_use=multi_use)

**BREAK POINT:** Please switch to agent `Authority`, open `01_issue_VC_city.ipynb`, and continue with Step 2

---

### 2.3 – Sent trust ping to establish and activate connection

## 3 – Request VC from `Authority` agent
### 3.1 – Message `Authority` to request a VC

In [ ]:
connections.send_message(connection_id, "testing connection")

In [ ]:
basic_message = "Hello Authority agent"
await agent_controller.messaging.send_message(connection_id, basic_message)

**BREAK POINT:** Go to Step 3 in the `01_issue_VC_city.ipynb` notebook of the `Authority` agent.

---

### 3.1 – Provide `Authority` with the relevant information to issue a VC

In [ ]:
basic_message = '{"manufacturerName": "undisclosedName1", "manufacturerCountry": "DE", "manufacturerCity": "Munich"}'
await agent_controller.messaging.send_message(connection_id, basic_message)

**BREAK POINT:** Return to step 

---


### 3.2 – Request VC from `Authority`'s Offer

Note: Your agent will automatically respond if ACAPY_AUTO_RESPOND_CREDENTIAL_OFFER=true flag is set in .env file of agent. Default is false.

To respond to an offer you must identify the offer using the credential_exchange_id generated for it. This is available from within the issue-credential holder handler. You could add custom logic in this loop `elif state == "request":` to handle this.

However, we will fetch the credential exchange records and **assume** this agent only has one record. Customise accordingly.

In [ ]:
identifiers = fnc.get_identifiers()
schema_id = identifiers["manufacturer_schema_identifiers"]["schema_id"]
schema_id

In [ ]:
connections.request_vc(connection_id, schema_id)

### 3.3 – Store received VC in wallet

This will be done automatically if the ACAPY_AUTO_STORE_CREDENTIAL=true flag is set in the .env file for this agent. Default is false.

Again you could handle this in your holder handler function in the `elif state == "credential_received":` loop.

In [ ]:
connections.get_credentials()

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()